In [8]:
import pandas as pd
pd.set_option("display.max_rows", None)
df = pd.read_excel("datasets/persona.xlsx")
print(df.head())
print("------------------------------------")
print("Boyut Bilgisi: ", df.shape)
print("-------------------------------------")
print(df.info())

   Unnamed: 0  PRICE   SOURCE   SEX COUNTRY  AGE
0           0     39  android  male     bra   17
1           1     39  android  male     bra   17
2           2     49  android  male     bra   17
3           3     29  android  male     tur   17
4           4     49  android  male     tur   17
------------------------------------
Boyut Bilgisi:  (5000, 6)
-------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  5000 non-null   int64 
 1   PRICE       5000 non-null   int64 
 2   SOURCE      5000 non-null   object
 3   SEX         5000 non-null   object
 4   COUNTRY     5000 non-null   object
 5   AGE         5000 non-null   int64 
dtypes: int64(3), object(3)
memory usage: 234.5+ KB
None


In [23]:
df["SOURCE"].nunique()

2

In [24]:
df["SOURCE"].value_counts()

SOURCE
android    2974
ios        2026
Name: count, dtype: int64

In [25]:
df["PRICE"].nunique()

6

In [26]:
df["PRICE"].value_counts()

PRICE
29    1305
39    1260
49    1031
19     992
59     212
9      200
Name: count, dtype: int64

In [27]:
df["COUNTRY"].value_counts()

COUNTRY
usa    2065
bra    1496
deu     455
tur     451
fra     303
can     230
Name: count, dtype: int64

In [28]:
df.groupby("COUNTRY")["PRICE"].count()

COUNTRY
bra    1496
can     230
deu     455
fra     303
tur     451
usa    2065
Name: PRICE, dtype: int64

In [29]:
df.pivot_table(values="PRICE",index="COUNTRY",aggfunc="count")

,PRICE
COUNTRY,
bra,1496
can,230
deu,455
fra,303
tur,451
usa,2065


In [30]:
df.groupby("COUNTRY")["PRICE"].sum()

COUNTRY
bra    51354
can     7730
deu    15485
fra    10177
tur    15689
usa    70225
Name: PRICE, dtype: int64

In [31]:
df.groupby("COUNTRY").agg({"PRICE": "sum"})

,PRICE
COUNTRY,
bra,51354
can,7730
deu,15485
fra,10177
tur,15689
usa,70225


In [32]:
df.pivot_table(values="PRICE",index="COUNTRY",aggfunc="sum")

,PRICE
COUNTRY,
bra,51354
can,7730
deu,15485
fra,10177
tur,15689
usa,70225


In [33]:
df["SOURCE"].value_counts()

SOURCE
android    2974
ios        2026
Name: count, dtype: int64

In [34]:
# Soru 8: Ülkelere göre PRICE ortalamaları nedir?

In [35]:
df.groupby(by=['COUNTRY']).agg({"PRICE": "mean"})

,PRICE
COUNTRY,
bra,34.327540
can,33.608696
deu,34.032967
fra,33.587459
tur,34.787140
usa,34.007264


In [37]:
df.groupby(by=['SOURCE']).agg({"PRICE": "mean"})

,PRICE
SOURCE,
android,34.174849
ios,34.069102


In [40]:
df.groupby(by=["COUNTRY", 'SOURCE']).agg({"PRICE": "mean"})

PRICE
COUNTRY SOURCE            
bra     android  34.387029
        ios      34.222222
can     android  33.330709
        ios      33.951456
deu     android  33.869888
        ios      34.268817
fra     android  34.312500
        ios      32.776224
tur     android  36.229437
        ios      33.272727
usa     android  33.760357
        ios      34.371703

In [42]:
df.groupby(["COUNTRY", 'SOURCE', "SEX", "AGE"]).agg({"PRICE": "mean"}).head()

PRICE
COUNTRY SOURCE  SEX    AGE           
bra     android female 15   38.714286
                       16   35.944444
                       17   35.666667
                       18   32.255814
                       19   35.206897

In [43]:
agg_df = df.groupby(by=["COUNTRY", 'SOURCE', "SEX", "AGE"]).agg({"PRICE": "mean"}).sort_values("PRICE", ascending=False)
agg_df.head()

,,,,PRICE
COUNTRY,SOURCE,SEX,AGE,
bra,android,male,46,59.0
usa,android,male,36,59.0
fra,android,female,24,59.0
usa,ios,male,32,54.0
deu,android,female,36,49.0


In [44]:
agg_df = agg_df.reset_index()
agg_df.head()

,COUNTRY,SOURCE,SEX,AGE,PRICE
0,bra,android,male,46,59.0
1,usa,android,male,36,59.0
2,fra,android,female,24,59.0
3,usa,ios,male,32,54.0
4,deu,android,female,36,49.0


In [48]:
# AGE değişkeninin nerelerden bölüneceğini belirtelim:
bins = [0, 18, 23, 30, 40, agg_df["AGE"].max()]

In [49]:
# Bölünen noktalara karşılık isimlendirmelerin ne olacağını ifade edelim:
mylabels = ['0_18', '19_23', '24_30', '31_40', '41_' + str(agg_df["AGE"].max())]

In [50]:
# age'i bölelim:
agg_df["age_cat"] = pd.cut(agg_df["AGE"], bins, labels=mylabels)
agg_df.head()

,COUNTRY,SOURCE,SEX,AGE,PRICE,age_cat
0,bra,android,male,46,59.0,41_66
1,usa,android,male,36,59.0,31_40
2,fra,android,female,24,59.0,24_30
3,usa,ios,male,32,54.0,31_40
4,deu,android,female,36,49.0,31_40


In [53]:
# YÖNTEM 2
agg_df['customers_level_based'] = agg_df[['COUNTRY', 'SOURCE', 'SEX', 'age_cat']].agg(lambda x: '_'.join(x).upper(), axis=1)

In [54]:
# YÖNTEM 3
agg_df["customers_level_based"] = ['_'.join(i).upper() for i in agg_df.drop(["AGE", "PRICE"], axis=1).values]

In [55]:
[row[0].upper() + "_" + row[1].upper() + "_" + row[2].upper() + "_" + row[5].upper() for row in agg_df.values]

['BRA_ANDROID_MALE_41_66',
 'USA_ANDROID_MALE_31_40',
 'FRA_ANDROID_FEMALE_24_30',
 'USA_IOS_MALE_31_40',
 'DEU_ANDROID_FEMALE_31_40',
 'DEU_ANDROID_FEMALE_41_66',
 'DEU_IOS_MALE_19_23',
 'USA_ANDROID_FEMALE_41_66',
 'FRA_IOS_MALE_19_23',
 'USA_IOS_MALE_41_66',
 'DEU_ANDROID_MALE_0_18',
 'BRA_ANDROID_FEMALE_24_30',
 'USA_ANDROID_FEMALE_41_66',
 'TUR_IOS_MALE_24_30',
 'DEU_ANDROID_MALE_31_40',
 'TUR_IOS_FEMALE_0_18',
 'BRA_ANDROID_FEMALE_41_66',
 'BRA_IOS_FEMALE_19_23',
 'TUR_ANDROID_FEMALE_31_40',
 'BRA_ANDROID_FEMALE_31_40',
 'TUR_IOS_MALE_31_40',
 'TUR_ANDROID_MALE_0_18',
 'USA_IOS_MALE_24_30',
 'BRA_IOS_MALE_41_66',
 'DEU_ANDROID_MALE_19_23',
 'USA_IOS_MALE_24_30',
 'TUR_ANDROID_FEMALE_0_18',
 'USA_ANDROID_MALE_41_66',
 'TUR_ANDROID_MALE_24_30',
 'USA_IOS_FEMALE_41_66',
 'FRA_ANDROID_MALE_19_23',
 'USA_IOS_MALE_31_40',
 'TUR_ANDROID_FEMALE_0_18',
 'FRA_IOS_MALE_0_18',
 'TUR_IOS_FEMALE_19_23',
 'TUR_ANDROID_FEMALE_31_40',
 'TUR_ANDROID_MALE_24_30',
 'FRA_ANDROID_MALE_19_23',
 'BRA_AN

In [56]:
# Veri setine ekleyelim:
agg_df["customers_level_based"] = [row[0].upper() + "_" + row[1].upper() + "_" + row[2].upper() + "_" + row[5].upper() for row in agg_df.values]
agg_df.head()

,COUNTRY,SOURCE,SEX,AGE,PRICE,age_cat,customers_level_based
0,bra,android,male,46,59.0,41_66,BRA_ANDROID_MALE_41_66
1,usa,android,male,36,59.0,31_40,USA_ANDROID_MALE_31_40
2,fra,android,female,24,59.0,24_30,FRA_ANDROID_FEMALE_24_30
3,usa,ios,male,32,54.0,31_40,USA_IOS_MALE_31_40
4,deu,android,female,36,49.0,31_40,DEU_ANDROID_FEMALE_31_40


In [57]:
# Gereksiz değişkenleri çıkaralım:
agg_df = agg_df[["customers_level_based", "PRICE"]]
agg_df.head()

,customers_level_based,PRICE
0,BRA_ANDROID_MALE_41_66,59.0
1,USA_ANDROID_MALE_31_40,59.0
2,FRA_ANDROID_FEMALE_24_30,59.0
3,USA_IOS_MALE_31_40,54.0
4,DEU_ANDROID_FEMALE_31_40,49.0


In [58]:
for i in agg_df["customers_level_based"].values:
    print(i.split("_"))

['BRA', 'ANDROID', 'MALE', '41', '66']
['USA', 'ANDROID', 'MALE', '31', '40']
['FRA', 'ANDROID', 'FEMALE', '24', '30']
['USA', 'IOS', 'MALE', '31', '40']
['DEU', 'ANDROID', 'FEMALE', '31', '40']
['DEU', 'ANDROID', 'FEMALE', '41', '66']
['DEU', 'IOS', 'MALE', '19', '23']
['USA', 'ANDROID', 'FEMALE', '41', '66']
['FRA', 'IOS', 'MALE', '19', '23']
['USA', 'IOS', 'MALE', '41', '66']
['DEU', 'ANDROID', 'MALE', '0', '18']
['BRA', 'ANDROID', 'FEMALE', '24', '30']
['USA', 'ANDROID', 'FEMALE', '41', '66']
['TUR', 'IOS', 'MALE', '24', '30']
['DEU', 'ANDROID', 'MALE', '31', '40']
['TUR', 'IOS', 'FEMALE', '0', '18']
['BRA', 'ANDROID', 'FEMALE', '41', '66']
['BRA', 'IOS', 'FEMALE', '19', '23']
['TUR', 'ANDROID', 'FEMALE', '31', '40']
['BRA', 'ANDROID', 'FEMALE', '31', '40']
['TUR', 'IOS', 'MALE', '31', '40']
['TUR', 'ANDROID', 'MALE', '0', '18']
['USA', 'IOS', 'MALE', '24', '30']
['BRA', 'IOS', 'MALE', '41', '66']
['DEU', 'ANDROID', 'MALE', '19', '23']
['USA', 'IOS', 'MALE', '24', '30']
['TUR', 'AN

In [59]:
agg_df["customers_level_based"].value_counts()

customers_level_based
BRA_ANDROID_MALE_24_30      7
USA_ANDROID_MALE_41_66      7
USA_IOS_FEMALE_24_30        7
BRA_ANDROID_FEMALE_24_30    7
USA_ANDROID_MALE_24_30      7
BRA_IOS_MALE_31_40          6
USA_ANDROID_FEMALE_24_30    6
USA_IOS_MALE_24_30          6
BRA_IOS_FEMALE_24_30        6
USA_ANDROID_MALE_31_40      6
BRA_ANDROID_FEMALE_31_40    6
USA_IOS_FEMALE_41_66        5
BRA_ANDROID_FEMALE_19_23    5
USA_ANDROID_FEMALE_31_40    5
TUR_IOS_FEMALE_19_23        5
BRA_IOS_MALE_41_66          5
BRA_ANDROID_MALE_19_23      5
USA_ANDROID_MALE_19_23      5
USA_ANDROID_FEMALE_19_23    5
DEU_ANDROID_MALE_31_40      5
BRA_IOS_FEMALE_31_40        5
USA_IOS_MALE_41_66          5
USA_ANDROID_FEMALE_41_66    5
USA_IOS_MALE_31_40          5
USA_IOS_FEMALE_19_23        5
DEU_IOS_FEMALE_0_18         4
USA_IOS_FEMALE_0_18         4
TUR_ANDROID_FEMALE_41_66    4
BRA_ANDROID_MALE_41_66      4
BRA_IOS_MALE_0_18           4
BRA_IOS_MALE_19_23          4
BRA_IOS_FEMALE_0_18         4
DEU_ANDROID_FEMALE

In [60]:
# Bu sebeple segmentlere göre groupby yaptıktan sonra price ortalamalarını almalı ve segmentleri tekilleştirmeliyiz.
agg_df = agg_df.groupby("customers_level_based").agg({"PRICE": "mean"})

In [61]:
# customers_level_based index'te yer almaktadır. Bunu değişkene çevirelim.
agg_df = agg_df.reset_index()
agg_df.head()


,customers_level_based,PRICE
0,BRA_ANDROID_FEMALE_0_18,35.645303
1,BRA_ANDROID_FEMALE_19_23,34.077340
2,BRA_ANDROID_FEMALE_24_30,33.863946
3,BRA_ANDROID_FEMALE_31_40,34.898326
4,BRA_ANDROID_FEMALE_41_66,36.737179


In [62]:
# kontrol edelim. her bir persona'nın 1 tane olmasını bekleriz:
agg_df["customers_level_based"].value_counts()
agg_df.head()

,customers_level_based,PRICE
0,BRA_ANDROID_FEMALE_0_18,35.645303
1,BRA_ANDROID_FEMALE_19_23,34.077340
2,BRA_ANDROID_FEMALE_24_30,33.863946
3,BRA_ANDROID_FEMALE_31_40,34.898326
4,BRA_ANDROID_FEMALE_41_66,36.737179


In [66]:
agg_df["SEGMENT"] = pd.qcut(agg_df["PRICE"], 4, labels=["D", "C", "B", "A"])

In [67]:
agg_df.head(30)

,customers_level_based,PRICE,SEGMENT
0,BRA_ANDROID_FEMALE_0_18,35.645303,B
1,BRA_ANDROID_FEMALE_19_23,34.077340,C
2,BRA_ANDROID_FEMALE_24_30,33.863946,C
3,BRA_ANDROID_FEMALE_31_40,34.898326,B
4,BRA_ANDROID_FEMALE_41_66,36.737179,A
5,BRA_ANDROID_MALE_0_18,34.805861,B
6,BRA_ANDROID_MALE_19_23,31.673243,D
7,BRA_ANDROID_MALE_24_30,33.413919,C
8,BRA_ANDROID_MALE_31_40,34.327381,B
9,BRA_ANDROID_MALE_41_66,40.041667,A


In [68]:
agg_df.groupby("SEGMENT").agg({"PRICE": "mean"})

,PRICE
SEGMENT,
D,29.206780
C,33.509674
B,34.999645
A,38.691234


In [69]:
new_user = "TUR_ANDROID_FEMALE_31_40"
agg_df[agg_df["customers_level_based"] == new_user]

,customers_level_based,PRICE,SEGMENT
72,TUR_ANDROID_FEMALE_31_40,41.833333,A


In [70]:
new_user = "FRA_IOS_FEMALE_31_40"
agg_df[agg_df["customers_level_based"] == new_user]

,customers_level_based,PRICE,SEGMENT
63,FRA_IOS_FEMALE_31_40,32.818182,C
